In [21]:
import sys

from sndata.csp import dr1, dr3

sys.path.insert(0, '../')
from phot_class.spectra import tabulate_spectral_properties

dr1.download_module_data()
dr3.download_module_data()


Fetching http://cdsarc.u-strasbg.fr/viz-bin/nph-Cat/tar.gz?J/AJ/154/211
Fetching https://csp.obs.carnegiescience.edu/data/CSP_Photometry_DR3.tgz
Fetching https://csp.obs.carnegiescience.edu/data/u_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/g_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/r_tel_ccd_atm_ext_1.2_new.dat
Fetching https://csp.obs.carnegiescience.edu/data/i_tel_ccd_atm_ext_1.2_new.dat
Fetching https://csp.obs.carnegiescience.edu/data/B_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/V_LC3014_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/V_LC3009_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/V_tel_ccd_atm_ext_1.2.dat
Fetching https://csp.obs.carnegiescience.edu/data/Y_SWO_TAM_scan_atm.dat
Fetching https://csp.obs.carnegiescience.edu/data/J_old_retrocam_swope_atm.dat
Fetching https://csp.obs.carnegiescience.edu/data/J_SWO_TAM_atm.dat
Fetch

In [3]:
date, wave, flux = [], [], []
for spectra in dr1.iter_data():
    spectra = spectra.group_by('date')
    
    for spectrum in spectra.groups:
        date.append(spectrum['date'][0])
        wave.append(spectrum['wavelength'])
        flux.append(spectrum['flux'])
        
    break

In [6]:
tabulate_spectral_properties(date, wave, flux, spectra.meta['redshift'], spectra.meta['Ra'], spectra.meta['dec'])

KeyError: 'Ra'

In [7]:
spectra.meta

OrderedDict([('comments',
              ['FITS_file:            SN04dt_b01_CLA_LD_24oct04.fits',
               'Redshift:             0.01973',
               'JDate_of_max:         2453239.06',
               'JDate_of_observation: 2453302.77',
               'Epoch:                62.48',
               '',
               'FITS_file:            SN04dt_b01_DUP_WF_20sep04.fits',
               'Redshift:             0.01973',
               'JDate_of_max:         2453239.06',
               'JDate_of_observation: 2453268.72',
               'Epoch:                29.09',
               '',
               'FITS_file:            SN04dt_b01_DUP_WF_14sep04.fits',
               'Redshift:             0.01973',
               'JDate_of_max:         2453239.06',
               'JDate_of_observation: 2453262.71',
               'Epoch:                23.19',
               '',
               'FITS_file:            SN04dt_b01_DUP_WF_19sep04.fits',
               'Redshift:             0.01973